Notes:

1. You will have to install isce2 e.g. `mamba install -c conda-forge isce2`
2. You will have to launch this notebook from a terminal in which the environment is activated since this notebook sends commands to the terminal (specifically `isce2` commands) 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dem_stitcher.stitcher import stitch_dem
import rasterio
import matplotlib.pyplot as plt
from pathlib import Path
import subprocess
import site
from lxml import etree
import isce
from shapely.geometry import box
import numpy as np

Using default ISCE Path: /Users/cmarshak/opt/anaconda3/envs/dem_stitcher/lib/python3.8/site-packages/isce
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce


# Parameters

In [3]:
dem_name = 'glo_30'
bounds = [-169.0, 53., -167.0, 54.]

In [4]:
def tag_dem_xml_as_ellipsoidal(dem_path: Path) -> str:
    xml_path = str(dem_path) + '.xml'
    assert(Path(xml_path).exists())
    tree = etree.parse(xml_path)
    root = tree.getroot()

    y = etree.Element("property", name='reference')
    etree.SubElement(y, "value").text = "WGS84"
    etree.SubElement(y, "doc").text = "Geodetic datum"

    root.insert(0, y)
    with open(xml_path, 'wb') as file:
        file.write(etree.tostring(root, pretty_print=True))
    return xml_path


def fix_image_xml(isce_raster_path: str) -> str:
    isce_apps_path = site.getsitepackages()[0] + '/isce/applications'
    fix_cmd = [f'{isce_apps_path}/fixImageXml.py',
               '-i',
               str(isce_raster_path),
               '--full']
    fix_cmd_line = ' '.join(fix_cmd)
    subprocess.check_call(fix_cmd_line, shell=True)
    return isce_raster_path


def download_dem_for_isce2(extent: list,
                           dem_name: str = 'glo_30',
                           dem_dir: Path = None,
                           buffer: float = .004) -> dict:
    """
    Parameters
    ----------
    extent : list
        [xmin, ymin, xmax, ymin] for epsg:4326 (i.e. (x, y) = (lon, lat))
    dem_name : str, optional
        See names in `dem_stitcher`
    full_res_dem_dir : Path, optional
    low_res_dem_dir : Path, optional
    buffer : float, optional
        In degrees, by default .004, which is .5 km at equator
    Returns
    -------
    Path
    """
    dem_dir = dem_dir or Path(f'{dem_name}')
    dem_dir.mkdir(exist_ok=True, parents=True)

    extent_geo = box(*extent)
    extent_buffered = list(extent_geo.buffer(buffer).bounds)
    extent_buffered = list(map(lambda e: round(e, 3), extent_buffered))

    dem_array, dem_profile = stitch_dem(extent_buffered,
                                        dem_name,
                                        dst_ellipsoidal_height=True,
                                        max_workers=5)

    full_res_dem_path = dem_dir/f'{dem_name}.dem.wgs84'
    dem_array[np.isnan(dem_array)] = 0.
    dem_profile['nodata'] = None
    dem_profile['driver'] = 'ISCE'
    with rasterio.open(full_res_dem_path, 'w', **dem_profile) as ds:
        ds.write(dem_array, 1)

    full_res_dem_xml = tag_dem_xml_as_ellipsoidal(full_res_dem_path)

    fix_image_xml(full_res_dem_xml)

    return full_res_dem_xml

In [5]:
download_dem_for_isce2(extent=bounds,
                       dem_name=dem_name,
                       dem_dir=Path('isce_dem'))

Reading glo_30 Datasets: 100%|██████████| 12/12 [00:09<00:00,  1.21it/s]


'isce_dem/glo_30.dem.wgs84.xml'